<a href="https://colab.research.google.com/github/tdmpni/coursera_capstone1/blob/main/Coursera_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LeNike Pop-up Store Evaluation
*This is a project for the IBM Data Science certification*

## Introduction & Business Problem

Pop up stores are a great marketing tool for various brands and particularly for those that do not traditionally operate brick and mortar stores.  
A pop up store is where a brand opens a store in a certain area  for a short period of time. The likes of Nike, BarkShop and many more have had great success with pop up stores. Pop up stores can lead to massive social media exposure and rapid sales as people have a fear of missing out since they know the store will be gone at any moment.

**The problem:** _Where an online brand should open a pop-up store_

Toronto is North America's fastest growing city, and this project shall tackle the business problem of leading a online clothing brand deciding on where in Toronto to open a pop-up store. 

The brand shall be called LeNike. The ideal location will be trendy, and have a high concentration of other retail businesses to ensure a high amount of foot traffic.

Commercial real estate in Toronto is some of the most expensive in North America, so LeNike needs to make sure it picks the right spot for their investment to pay off. 

The stakeholders for this project are LeNike's executive team that will need to be presented with this data in order to decide where in Toronto to open the store. 

The executives must ideally pick the best location as 2020 was a difficult year for retail, and they need to have a successful 2021 in order for the company to continue surviving. 
